# Homeworks - Big Data and Public Policy Class

In [ ]:
from IPython.display import display
import pandas as pd
pd.options.display.max_columns = None # Display all columns of a dataframe
pd.options.display.max_rows = 700
from pprint import pprint
import re

The objective of the following homework is to **bring to practice the tools studied in class** by conducting step by step a first big data project. We advise that you do it in parallel with the lectures: the relation between the project steps and the lectures is indicated below. 

You will build several **machine learning models that will predict continuous and categorical variables based on data coming from webscraping and/or an API**. 

We propose that you predict the **daily stock price data of a large company** (of your choice) **based on the company's tweets**. You can chose a company from one of the [major stock indices](https://www.wikiwand.com/en/List_of_stock_market_indices). 

---------------------------
After having decided the setting (your own or the company + corresponding timespan) that you will study, you should follow the following steps:

**Step 1a: `X` variables** [week 2]
- Fetch the data using the **twitter API** or any other API or website that you are interested in.
- Beware of the rate limits and organize your program so as to overcome them if needed
- The data should include some text, but might also have other interesting variables (retweets, favorites...)
- Create some (non-text based) `X_num` variables that you will use for the prediction

In [3]:
#this you get when you make create an application on twitter as a dev
consumer_key="7X8q1LteL1qOLRg4DSoiI0lyk"
consumer_secret="Bb08vV5XoxDEP4SGLJfkvEpuwvxEDvzVSRGTptzQCjC9XVmquP"
access_token="1230145588659998721-EQb8DoMYnorBtqAuEPMZtZcXumzuAe"
access_token_secret="mzurpOu1LWhj0NU9pctgYZ6OBzhQvyqw8hvRNpe3yI4Qc"

## Accessing the Twitter API using `tweepy`

We use the `tweepy` package (documentation: https://tweepy.readthedocs.io/en/latest/). Tweepy is an *An easy-to-use Python library for accessing the Twitter API.*

R users can use [rtweet](https://rtweet.info/), a similar package. 

Twitter requires all requests to use `OAuth` for authentication

In [4]:
import tweepy
from tweepy import OAuthHandler

Authenticate to Twitter

In [5]:
auth = OAuthHandler(consumer_key, consumer_secret) #creating an OAuthHandler instance
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

We specify `wait_on_rate_limit_notify==True` & `wait_on_rate_limit=True`. The API method will wait once you’ve reached your rate limit and prints out a message

In [6]:
# test authentication
try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


For an extensive list of the methods available, see the [API Reference page](https://tweepy.readthedocs.io/en/latest/api.html#api-reference). There are several types of methods. The following methods enable you to access twitter content:
- Timeline methods return a list of  `status` objects
- Status methods return a `status` object
- User methods return `user` object or a list of `user` objects. 
- Favorite methods: return a list of  `status` objects

For some methods, you can interact with twitter:
- Friendship Methods return a `user` object, by example:
    - `create_friendship`: creates a new friendship with the specified user ()
    
Let's review some useful methods:

### Methods returning a `status` object (or a list of objects)
#### Search method
If you seeking Twitter data to get conversations on a particular topic. This method returns a collection of relevant Tweets matching a specified query for all public tweets.

In [7]:
# most recent tweets about ETH 
tweets = api.search(q="ETH Zürich", lang="en") # TODO: How is it set to recent tweets?
for tweet in tweets:
    print(tweet.text) # printing the first tweet
    #todo still not working 15:30

RT @_onurmutlu_: Computer Architecture - Lecture 21a: Multiprocessing (ETH Zürich, Fall 2019)
https://t.co/qsjRv6eV4R via @YouTube @CSatETH…
Andreas Krause (ETH Zuerich): &amp;quot;Safe Exploration in Reinforcement Learning&amp;quot;
https://t.co/ZAsy9yM0vO
#reinforcementlearning
RT @DuaneKing: Franz Max Osswald, contact print of sound photographs in architectural models, from Osswald’s applied acoustics laboratory a…
RT @mikarv: Official PEPP-PT severance notice from ETH Zürich, following PEPP-PT's failure to publish *any* documents or protocol, as they…
Watch "H.H.Dalai Lama and Matthieu Ricard. "Fear and anxiety". ETH Zürich 2005. By Guido Ferrari" on YouTube https://t.co/KCgauwUm7V
A study by ETH Zürich shows that before lockdown was put in place on 16 March, each person infected with COVID-19 w… https://t.co/mWmS2RSRqn
@biogogy Went to a great symposium a decade ago at ETH Zürich. Called Modern Biology goes to School. Uni Bio and Bi… https://t.co/KDOF5JFaIl
Thanks to our amazing p

The `status` object:

In [8]:
#todo: check to do list not working
# todo: add this
pprint(tweets[0]) # for the first tweet

Status(_api=<tweepy.api.API object at 0x0000028BED9D6448>, _json={'created_at': 'Sat Apr 25 08:25:44 +0000 2020', 'id': 1253963392542531586, 'id_str': '1253963392542531586', 'text': 'RT @_onurmutlu_: Computer Architecture - Lecture 21a: Multiprocessing (ETH Zürich, Fall 2019)\nhttps://t.co/qsjRv6eV4R via @YouTube @CSatETH…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': '_onurmutlu_', 'name': 'Onur Mutlu', 'id': 12309242, 'id_str': '12309242', 'indices': [3, 15]}, {'screen_name': 'YouTube', 'name': 'YouTube at 🏠', 'id': 10228272, 'id_str': '10228272', 'indices': [122, 130]}, {'screen_name': 'CSatETH', 'name': 'ETH CS Department', 'id': 2819604519, 'id_str': '2819604519', 'indices': [131, 139]}], 'urls': [{'url': 'https://t.co/qsjRv6eV4R', 'expanded_url': 'https://youtu.be/3o2WieJsAhw', 'display_url': 'youtu.be/3o2WieJsAhw', 'indices': [94, 117]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="h

In [9]:
pprint(tweets[0]._api) # for the first tweet

In [10]:
pprint(tweets[0]._json) # for the first tweet

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sat Apr 25 08:25:44 +0000 2020',
 'entities': {'hashtags': [],
              'symbols': [],
              'urls': [{'display_url': 'youtu.be/3o2WieJsAhw',
                        'expanded_url': 'https://youtu.be/3o2WieJsAhw',
                        'indices': [94, 117],
                        'url': 'https://t.co/qsjRv6eV4R'}],
              'user_mentions': [{'id': 12309242,
                                 'id_str': '12309242',
                                 'indices': [3, 15],
                                 'name': 'Onur Mutlu',
                                 'screen_name': '_onurmutlu_'},
                                {'id': 10228272,
                                 'id_str': '10228272',
                                 'indices': [122, 130],
                                 'name': 'YouTube at 🏠',
                                 'screen_name': 'YouTube'},
                                {'id': 2819604519,
  

**Step 1b: continuous `y` variables** [week 2]
- Fetching the data: 
    - if you work on the suggested idea, you can easily access daily stock prices using the [`yfinance` package](https://pypi.org/project/yfinance/)  (see below)
    - otherwise, you can find some interesting data listed in the syllabus

**Step 1c: merge `X_num` and `y`** [week 2]
- Beware of the temporality: in the case of the proposed study on stock market prices, you will have to deal with the fact that the X is at the tweet level while `y` is daily. 

***Milestone 1 - March 24th*** *You can submit the previous steps as a first notebook.*

**Sample split.** Do the standard 80% / 20% training/test split using all days in the data. In addition, do a separate temporal split where the training set is the first 80% of days in the time series.

**For all machine learning models**, report performance measure in test and train samples.

**Step 2a: estimate different regression models using `X_num` and `y`** [week 3]

***Milestone 2 - March 31th***  *You can submit the previous steps as a second notebook.*

**Step 3: text analysis** [week 4]
- Featurize tweets (or another text dataset related to your subject): transform the text into a standard document-level dataset `X_doc`

**Step 2b: estimate different regression models using `X_doc` and `y`** [week 3]

**Step 4: estimate a classification models** [week 5]
- propose a categorical variable `y_calc` that you can compute from the continuous one (`y`) (e.g. positive or negative growthin stock prices). For the `X` dimension, you can use `X_doc` or `X_num` or both. 
- you can use any other categorical variable that you find relevant 

***Milestone 3 - April 21th***  *You can submit the previous steps as a third notebook.*

**Step 6: Dimension reduction** [week 6]
- Use one of the dimension reductions methods to dimension-reduce the features
    - PCA or topic model (LDA or STM) or k-means clustering on the featurized text `X_doc`
- Run another classifier

**Step bonus: Econometric identification** [week 8]
- Find an exogenous shock affecting this firm (but not all the firms) and a control group of firms not affected
    - example: a natural disaster/shock to the exchange rate/change in ownership... affects the functionning of this firm but not the other firms of the stock market index.  
- scale up the previous data collection to the firms in the control group
- use one of the technique studied in class to causaly identify the impact of the exogenous shock on the stock market of the affected firm

***Milestone 4 - May 17th*** *You can submit the previous steps as a last notebook.*

---------------------------
**Requirement for completion grade**: 
- the homework should be done in **groups of 1 or 2 persons** 
- the homework will have to be submitted as a (Python or R) **notebooks**
- you can give them back at the indicated **milestone** or on May 17th (no homework will be accepted after May 17th)
- the notebooks should contain at least **three graphs** (overall)
- each notebook should run from the beginning to then end, but:
    - beware of paths to folders that are on your computer but not on mine
    - if you have such a path, you can: 
        - have the data downloaded in the notebook directly
        - import your data in a github (or any other online storage) folder that the notebook can access  
    - indicate if the notebook takes more than 30 minutes to run
- do not write us emails regarding the homework, ask your questions on the **forum**, it will benefit everyone for sure! 

---------------------------

**Alternative homework**

If you prefer, we propose for two persons at most to translate all python notebooks of the class into R. This work would count as your homework for the class. Contact us in advance if you want to do this.

In [11]:
# Import the yfinance. 
# If you get module not found error the run !pip install yfinance from your Jupyter notebook
import yfinance as yf

ModuleNotFoundError: No module named 'yfinance'